In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import data_collecter_sohu as sohu_dc
pd.set_option('display.width',180)  
df_002024=sohu_dc.get_hist_data('002024','20080101','20171207')
df_002024_norm=(df_002024-df_002024.min())/(df_002024.max()-df_002024.min())

In [2]:
features_placeholder = tf.placeholder(tf.float32, shape=(None,85))

In [3]:
def prediction():                                                                              
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([85, 70],stddev=1.0 / math.sqrt(float(85))),name='weights')
        biases = tf.Variable(tf.zeros([70]),name='biases')
        hidden1 = tf.nn.tanh(tf.matmul(features_placeholder, weights) + biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([70, 20],stddev=1.0 / math.sqrt(float(70))),name='weights')
        biases = tf.Variable(tf.zeros([20]),name='biases')
        hidden2 = tf.nn.tanh(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.truncated_normal([20, 2],stddev=1.0 / math.sqrt(float(20))),name='weights')
        biases = tf.Variable(tf.zeros([2]),name='biases')
        logits = tf.matmul(hidden2, weights) + biases
    return logits

In [4]:
predic=prediction()
sess=tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [5]:
c1=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
id(c1[0])

140297525861568

In [6]:
saver.restore(sess,'/tmp/tensorflow/stock/logs/fully_connected_feed/model.ckpt-399999')

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/stock/logs/fully_connected_feed/model.ckpt-399999


In [7]:
c1=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
id(c1[0])

140297525861568

In [8]:
sess.run(c1[0])

array([[ 0.1401061 ,  0.16084856, -0.12736204, ..., -0.08255085,
        -0.03620422, -0.07942498],
       [-0.18311132,  0.06205653, -0.07545811, ...,  0.08592571,
         0.06185616, -0.00533621],
       [-0.02581165,  0.14213564, -0.00144491, ...,  0.07245395,
        -0.02809707,  0.13461024],
       ..., 
       [ 0.02144039,  0.03965239, -0.13567285, ..., -0.04471184,
        -0.09493538, -0.01244642],
       [-0.17776906,  0.10996253,  0.0520594 , ...,  0.00357649,
        -0.10422485, -0.16534671],
       [-0.10436725, -0.07790634,  0.00323875, ..., -0.00244721,
         0.02937283,  0.04878408]], dtype=float32)

In [9]:
sess.run(c1[4])

array([[ 0.04699164,  0.07830928],
       [-0.10488123, -0.12906377],
       [-0.49583456,  0.34723592],
       [-0.3829906 ,  0.4987368 ],
       [ 0.47979558, -0.17302161],
       [-0.51377118,  0.71264064],
       [-0.13888238, -0.14886467],
       [-2.57917237,  2.50576448],
       [ 3.6830287 , -3.87498713],
       [ 0.03048772,  0.00877799],
       [-0.045543  , -0.01523578],
       [ 0.13236479,  0.08163621],
       [-1.835096  ,  2.13054013],
       [-0.01261043, -0.21460834],
       [-0.71412182,  0.41640899],
       [-1.53054154,  1.85977578],
       [ 0.01038444, -0.01030469],
       [-0.02752909, -0.06467877],
       [ 0.19760621,  0.16097853],
       [ 0.89360982, -0.90749574]], dtype=float32)

In [10]:
data=df_002024_norm['20171207':'20080101']
features=[]
lables=[]
date=[]
for index in range(data.index.size):
    slice=data[data.index.size-6-index:data.index.size-index]
    if(slice.index.size<6):
        break
    features.append(slice[1:6].values.flatten())
    date.append(slice.index[0])
    if (slice.close[0]-slice.close[1])/slice.close[1]>0.03:
        lables.append(0)
    else:
        lables.append(1)


/home/tr/bin/anaconda2/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [11]:
df_002024.loc['20170928']

open                         12.47
close                        13.09
price_change                  0.69
p_change_rate                 5.56
low                          12.39
high                         13.13
volume                  1152831.00
amount                   148071.89
turnover                      2.31
index_open                11029.47
index_close               11031.42
index_price_change            5.36
index_p_change_rate           0.05
index_low                 11015.48
index_high                11072.56
index_volume           97338744.00
index_amount           12849238.00
Name: 20170928, dtype: float64

In [20]:
features=[]
lables=[]
date=[]
print(df_002024[df_002024.p_change_rate>3].index)
for i in df_002024.index:
    slice=data[df_002024.index.get_loc(i):df_002024.index.get_loc(i)+6]
    if(slice.index.size<6):
        break
    features.append(slice[1:6].values.flatten())
    date.append(slice.index[0])
print(date)
    
    
    

Index(['20171206', '20171113', '20171106', '20171102', '20171026', '20171019', '20171016', '20171009', '20170928', '20170920',
       ...
       '20080310', '20080303', '20080225', '20080218', '20080213', '20080204', '20080131', '20080123', '20080122', '20080116'],
      dtype='object', name='date', length=523)
['20171207', '20171206', '20171205', '20171204', '20171201', '20171130', '20171129', '20171128', '20171127', '20171124', '20171123', '20171122', '20171121', '20171120', '20171117', '20171116', '20171115', '20171114', '20171113', '20171110', '20171109', '20171108', '20171107', '20171106', '20171103', '20171102', '20171101', '20171031', '20171030', '20171027', '20171026', '20171025', '20171024', '20171023', '20171020', '20171019', '20171018', '20171017', '20171016', '20171013', '20171012', '20171011', '20171010', '20171009', '20170929', '20170928', '20170927', '20170926', '20170925', '20170922', '20170920', '20170919', '20170918', '20170915', '20170914', '20170913', '20170912', '2

In [21]:
a=sess.run(predic,{features_placeholder:features})


In [22]:
for i in range(a.size):
    y = np.exp(a[i])
    y1=y[0]/y.sum()
    y2=y[1]/y.sum()
    print(date[i],y1,y2)

20171207 0.990185 0.009815
20171206 0.995145 0.00485534
20171205 0.988424 0.011576
20171204 0.988035 0.0119646
20171201 0.997088 0.00291247
20171130 0.669272 0.330728
20171129 0.999545 0.000455352
20171128 0.938336 0.0616644
20171127 0.998309 0.00169071
20171124 0.956931 0.0430688
20171123 0.992782 0.0072181
20171122 0.99036 0.00963998
20171121 0.998414 0.0015858
20171120 0.997157 0.00284321
20171117 0.997463 0.00253754
20171116 0.998981 0.0010188
20171115 0.999756 0.000244161
20171114 0.97937 0.02063
20171113 0.996224 0.00377551
20171110 0.997589 0.00241085
20171109 0.998683 0.00131735
20171108 0.999007 0.000993277
20171107 0.998645 0.00135532
20171106 0.997246 0.00275391
20171103 0.996136 0.00386445
20171102 0.999624 0.000375801
20171101 0.99689 0.00310956
20171031 0.99951 0.000490474
20171030 0.998919 0.00108096
20171027 0.995231 0.00476946
20171026 0.99907 0.000929799
20171025 0.999033 0.000967345
20171024 0.999209 0.000790848
20171023 0.998976 0.00102424
20171020 0.99564 0.0043601

20140121 0.75947 0.24053
20140120 0.983634 0.0163656
20140117 0.893968 0.106032
20140116 0.938275 0.0617249
20140115 0.987428 0.0125716
20140114 0.894478 0.105522
20140113 0.797437 0.202563
20140110 0.990903 0.00909649
20140109 0.920144 0.0798557
20140108 0.870146 0.129854
20140107 0.98683 0.0131704
20140106 0.262659 0.737341
20140103 0.633797 0.366203
20140102 0.725454 0.274546
20131231 0.955631 0.044369
20131230 0.985983 0.0140167
20131227 0.488326 0.511674
20131226 0.987328 0.0126717
20131225 0.922507 0.077493
20131224 0.99768 0.00231993
20131223 0.944066 0.0559341
20131220 0.962739 0.0372607
20131219 0.974373 0.0256271
20131218 0.999075 0.000925405
20131217 0.948964 0.0510359
20131216 0.95706 0.0429405
20131213 0.989017 0.010983
20131212 0.999004 0.000995999
20131211 0.873637 0.126363
20131210 0.955445 0.0445554
20131209 0.859781 0.140219
20131206 0.853788 0.146212
20131205 0.987942 0.0120578
20131204 0.999751 0.00024855
20131203 0.817043 0.182957
20131202 0.990941 0.00905911
20131

20100310 0.99946 0.000539544
20100309 0.999096 0.000903737
20100308 0.999704 0.000295787
20100305 0.996866 0.00313371
20100304 0.997195 0.00280521
20100303 0.996813 0.00318733
20100302 0.995367 0.00463316
20100301 0.999296 0.000704015
20100226 0.998626 0.00137379
20100225 0.999839 0.000160648
20100224 0.998977 0.00102311
20100223 0.999181 0.000818876
20100222 0.998091 0.00190922
20100212 0.996333 0.00366742
20100211 0.99976 0.000240478
20100210 0.988221 0.0117787
20100209 0.999245 0.000754545
20100208 0.986715 0.0132847
20100205 0.99988 0.000120032
20100204 0.998924 0.00107579
20100203 0.999544 0.000455533
20100202 0.995778 0.00422245
20100201 0.999338 0.00066163
20100129 0.997077 0.00292347
20100128 0.999727 0.00027305
20100127 0.991672 0.00832823
20100126 0.999976 2.36956e-05
20100125 0.995325 0.00467547
20100122 0.999935 6.47098e-05
20100121 0.997963 0.00203654
20100120 0.99615 0.00385019
20100119 0.99598 0.00401953
20100118 0.999566 0.000433833
20100115 0.999778 0.00022181
20100114

IndexError: index 2382 is out of bounds for axis 0 with size 2382

In [ ]:
sess.run(c1[0])

In [ ]:
sess.run(c1[1])

In [ ]:
sess.run(c1[2])

In [ ]:
sess.run(c1[3])

In [ ]:
sess.run(c1[4])

In [ ]:
sess.run(c1[5])

In [ ]:
id(c1)

In [ ]:
saver.restore(sess,'/tmp/tensorflow/stock/logs/fully_connected_feed/model.ckpt-399999')
c1=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
id(c1)